Dataloaders

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install torchvision
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.9/960.9 kB 18.1 MB/s eta 0:00:00


### DataLoaders

In [33]:
import os
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


def create_dataloaders(train_dir, val_dir, test_dir, batch_size=32):
    """
    Create DataLoaders for train, validation, and test datasets.

    Args:
        train_dir (str): Path to the training directory with augmented images.
        val_dir (str): Path to the validation directory.
        test_dir (str): Path to the test directory.
        batch_size (int): Batch size for DataLoader.

    Returns:
        train_loader, val_loader, test_loader, class_names
    """
    #transforms
    train_transforms = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])  # normalize images to mean=0.5, std=0.5
    ])

    test_transforms = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

    # load datasets
    train_dataset = ImageFolder(root=train_dir, transform=train_transforms)
    val_dataset = ImageFolder(root=val_dir, transform=test_transforms)
    test_dataset = ImageFolder(root=test_dir, transform=test_transforms)

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, train_dataset.classes

### Training / Validation

In [34]:
import torch
import tqdm
from torch.utils.tensorboard import SummaryWriter
from torchmetrics import F1Score

# Configuration
save_model_path = "checkpoints/"
pth_name = "saved_model.pth"
os.makedirs(save_model_path, exist_ok=True)

def val(model, val_loader, loss_function, writer, epoch, device):
    f1 = F1Score(num_classes=len(val_loader.dataset.classes),average='weighted', task='multiclass')
    val_iterator = enumerate(val_loader)
    f1_list, f1t_list = [], []
    total_loss = 0
    total_correct = 0
    total_samples = 0

    model.eval()
    tq = tqdm.tqdm(total=len(val_loader), desc="Validation")
    with torch.no_grad():
        for _, (images, labels) in val_iterator:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            predictions = model(images)
            loss = loss_function(predictions, labels.long())  # Change labels to .long()
            total_loss += loss.item()

            # Compute predictions
            #predictions = predictions.softmax(dim=1)
            predictions = torch.argmax(predictions, dim=1)
            f1_list.extend(predictions.cpu().numpy())
            f1t_list.extend(labels.cpu().numpy())
            # Calculate accuracy
            #_, predicted = torch.max(predictions, 1)
            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)


            tq.update(1)

    tq.close()

    # Calculate F1 Score
    f1_score_value = f1(torch.tensor(f1_list), torch.tensor(f1t_list))
    val_accuracy = total_correct / total_samples

    writer.add_scalar("Validation F1", f1_score_value, epoch)
    writer.add_scalar("Validation Loss", total_loss / len(val_loader), epoch)
    writer.add_scalar("Validation Accuracy", val_accuracy, epoch)

    #print(f'list_pred {f1_list}')
    #print(f'list_pred {f1t_list}')
    print(f"Validation Loss: {total_loss / len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {f1_score_value:.4f}")



def train(model, train_loader, val_loader, optimizer, loss_fn, n_epochs, device):
    """
    Train the model on the training dataset.
    Logs loss and saves the model at each epoch.

    Args:
        model: PyTorch model.
        train_loader: DataLoader for training data.
        val_loader: DataLoader for validation data.
        optimizer: Optimizer for training.
        loss_fn: Loss function.
        n_epochs: Number of epochs.
        device: Device for computation (CPU/GPU).
    """
    writer = SummaryWriter()

    model.to(device)
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        total_correct = 0
        total_samples = 0
        f1 = F1Score(num_classes=len(train_loader.dataset.classes),average='weighted', task='multiclass').to(device)  # Move F1 metric to device

        tq = tqdm.tqdm(total=len(train_loader), desc=f"Epoch {epoch + 1}/{n_epochs}")
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Zero the gradient
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = loss_fn(outputs, labels)

            # Backward pass and optimizer step
            loss.backward()
            optimizer.step()

            # Update running loss
            running_loss += loss.item()

            # Compute predictions
            predictions = torch.argmax(outputs, dim=1)

            # Update F1 score
            f1.update(predictions, labels)

            # Accuracy calculation
            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)

            tq.set_postfix(loss=loss.item())
            tq.update(1)

        tq.close()

        # Compute F1 score at the end of the epoch
        f1_score_value = f1.compute()

        # Calculate training accuracy
        train_accuracy = total_correct / total_samples
        epoch_loss = running_loss / len(train_loader)

        # Log training metrics
        writer.add_scalar("Training Loss", epoch_loss, epoch)
        writer.add_scalar("Training Accuracy", train_accuracy, epoch)
        writer.add_scalar("Training F1", f1_score_value, epoch)

        # Print training metrics
        print(f"Epoch [{epoch + 1}/{n_epochs}], Training Loss: {epoch_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Training F1: {f1_score_value:.4f}")

        # Validate the model
        val(model, val_loader, loss_fn, writer, epoch, device)

        # Save the model checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        torch.save(checkpoint, os.path.join(save_model_path, pth_name))
        print(f"Model saved at {save_model_path}{pth_name}")

### Test

In [35]:
import torch
from sklearn.metrics import accuracy_score, f1_score
import tqdm  # Correct import for tqdm
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

def test(model, test_data_path, device, batch_size=32, image_size=(128, 128), mean=[0.5], std=[0.5]):
    """
    Evaluates a trained model on a test dataset.

    Parameters:
    - model (torch.nn.Module): The model to evaluate.
    - test_data_path (str): Path to the test dataset.
    - device (torch.device): The device to run the evaluation on (CPU or CUDA).
    - batch_size (int, optional): The batch size for the DataLoader. Default is 32.
    - image_size (tuple, optional): The target image size for the input images. Default is (128, 128).
    - mean (list, optional): The mean for normalization. Default is [0.5].
    - std (list, optional): The std for normalization. Default is [0.5].

    Returns:
    - None: Prints the accuracy and F1 score of the model on the test set.
    """

    # Load test dataset with transformations
    test_transforms = transforms.Compose([
        transforms.Resize(image_size),  # Resize to match input size for the model
        transforms.ToTensor(),          # Convert PIL images to tensors
        transforms.Normalize(mean=mean, std=std)  # Normalize (same mean/std as training)
    ])

    test_dataset = ImageFolder(root=test_data_path, transform=test_transforms)
    class_names = test_dataset.classes  # Retrieve class names
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Set the model to evaluation mode
    model.eval()

    # Initialize variables to store true labels and predicted labels
    all_true_labels = []
    all_pred_labels = []

    # Evaluate the model
    with torch.no_grad():  # No need to compute gradients during inference
        for inputs, labels in tqdm.tqdm(test_loader, desc="Evaluating"):  # Use tqdm from the module
            # Move the inputs and labels to the device (GPU or CPU)
            inputs, labels = inputs.to(device), labels.to(device)

            # Get model predictions
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)  # Get the predicted class (index of max logit)

            # Append true and predicted labels to the lists
            all_true_labels.extend(labels.cpu().numpy())
            all_pred_labels.extend(preds.cpu().numpy())

    # Compute accuracy and F1 score
    accuracy = accuracy_score(all_true_labels, all_pred_labels)
    f1 = f1_score(all_true_labels, all_pred_labels, average='macro')  # Weighted F1 score

    # Print results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score (Macro): {f1:.4f}")


In [36]:
!ls drive/MyDrive/ai-ml-final-project/dataset/

test  train  validation


In [37]:
!pwd

/content


In [39]:
!mkdir -p output/logs output/models

In [42]:
!ls -l

total 20
drwxr-xr-x 2 root root 4096 Apr  6 08:41 checkpoints
drwx------ 6 root root 4096 Apr  6 08:27 drive
drwxr-xr-x 4 root root 4096 Apr  6 08:49 output
drwxr-xr-x 4 root root 4096 Apr  6 08:39 runs
drwxr-xr-x 1 root root 4096 Apr  3 13:37 sample_data


### Variables

In [43]:
train_dir = 'drive/MyDrive/ai-ml-final-project/dataset/train'
val_dir = 'drive/MyDrive/ai-ml-final-project/dataset/validation'
test_dir = 'drive/MyDrive/ai-ml-final-project/dataset/test'
log_dir = "output/logs"
model_dir = "output/models"    # Model save path
batch_size = 32
num_epochs = 15
device = torch.device("cuda")

### Initializing DataLoaders

In [44]:
train_loader, val_loader, test_loader, class_names = create_dataloaders(
    train_dir=train_dir,
    val_dir=val_dir,
    test_dir=test_dir,
    batch_size=batch_size
)

# Resnet18

## Resnet18 | Pretrained

In [45]:
import torch.nn as nn
import torchvision.models as models

def get_resnet18_pretrained(num_classes, transfer_learning=True):
    model = models.resnet18(pretrained=transfer_learning)
    if transfer_learning:
        for param in model.parameters():
            param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

### With SGD

In [46]:
# ResNet18 Training
print("Training ResNet18...")

# Initialize the ResNet18 model with the number of classes
resnet18_pretrained_sgd = get_resnet18_pretrained(num_classes=len(class_names)).to(device)

# Define optimizers
optimizer_sgd = torch.optim.SGD(resnet18_pretrained_sgd.parameters(), lr=0.0005, momentum=0.9)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=resnet18_pretrained_sgd,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_sgd,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(resnet18_pretrained_sgd.state_dict(), os.path.join(model_dir, "resnet18_pretrained_sgd.pth"))
print(f"ResNet18 model saved to {os.path.join(model_dir, 'resnet18_pretrained_sgd.pth')}")

# Test the model
print("\nTest score:")
test(resnet18_pretrained_sgd, test_data_path=test_dir, device=device)

Training ResNet18...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

Epoch 1/15: 100%|██████████| 90/90 [00:26<00:00,  3.34it/s, loss=0.598]


Epoch [1/15], Training Loss: 0.6298, Training Accuracy: 0.6540, Training F1: 0.6540



Validation: 100%|██████████| 6/6 [00:01<00:00,  5.82it/s]


Validation Loss: 0.5567, Validation Accuracy: 0.7500, Validation F1 Score: 0.7433
Model saved at checkpoints/saved_model.pth


...

Epoch [15/15], Training Loss: 0.3596, Training Accuracy: 0.8397, Training F1: 0.8397



Validation: 100%|██████████| 6/6 [00:01<00:00,  5.91it/s]


Validation Loss: 0.4733, Validation Accuracy: 0.7611, Validation F1 Score: 0.7571
Model saved at checkpoints/saved_model.pth
ResNet18 model saved to output/models/resnet18_pretrained_sgd.pth

Test score:



Evaluating: 100%|██████████| 7/7 [00:50<00:00,  7.15s/it]

Accuracy: 0.8080
F1 Score (Macro): 0.8063


### With Adam

In [47]:
resnet18_pretrained_adam = get_resnet18_pretrained(num_classes=len(class_names)).to(device)
optimizer_adam = torch.optim.Adam(resnet18_pretrained_adam.parameters(), lr=0.0001)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=resnet18_pretrained_adam,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_adam,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(resnet18_pretrained_adam.state_dict(), os.path.join(model_dir, "resnet18_pretrained_adam.pth"))
print(f"ResNet18 model saved to {os.path.join(model_dir, 'resnet18_pretrained_adam.pth')}")

# Test the model
print("\nTest score:")
test(resnet18_pretrained_adam, test_data_path=test_dir, device=device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

Epoch 1/15: 100%|██████████| 90/90 [00:15<00:00,  5.82it/s, loss=0.813]


Epoch [1/15], Training Loss: 0.7408, Training Accuracy: 0.5196, Training F1: 0.5179



Validation: 100%|██████████| 6/6 [00:01<00:00,  5.86it/s]


Validation Loss: 0.6469, Validation Accuracy: 0.6000, Validation F1 Score: 0.5996
Model saved at checkpoints/saved_model.pth



Epoch 2/15: 100%|██████████| 90/90 [00:15<00:00,  5.75it/s, loss=0.556]


...

Epoch [15/15], Training Loss: 0.4438, Training Accuracy: 0.8003, Training F1: 0.8003



Validation: 100%|██████████| 6/6 [00:01<00:00,  5.78it/s]


Validation Loss: 0.4457, Validation Accuracy: 0.7667, Validation F1 Score: 0.7617
Model saved at checkpoints/saved_model.pth
ResNet18 model saved to output/models/resnet18_pretrained_adam.pth

Test score:



Evaluating: 100%|██████████| 7/7 [00:01<00:00,  3.87it/s]

Accuracy: 0.7634
F1 Score (Macro): 0.7626


## Resnet18 not-pretrained

In [48]:
def get_resnet18_np(num_classes, transfer_learning=False):
    model = models.resnet18(pretrained=False)  # Do not load pre-trained weights
    '''if transfer_learning:
        for param in model.parameters():
            param.requires_grad = False  # Freeze all layers if transfer learning '''
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

### With SGD

In [49]:
# ResNet18 Training
print("Training ResNet18...")

# Initialize the ResNet18 model with the number of classes
resnet18_sgd = get_resnet18_np(num_classes=len(class_names)).to(device)

# Define optimizers
optimizer_sgd = torch.optim.SGD(resnet18_sgd.parameters(), lr=0.0005, momentum=0.9)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=resnet18_sgd,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_sgd,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(resnet18_sgd.state_dict(), os.path.join(model_dir, "resnet18_sgd.pth"))
print(f"ResNet18 model saved to {os.path.join(model_dir, 'resnet18_sgd.pth')}")

# Test the model
print("\nTest score:")
test(resnet18_sgd, test_data_path=test_dir, device=device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Training ResNet18...



Epoch 1/15: 100%|██████████| 90/90 [00:19<00:00,  4.62it/s, loss=0.426]


Epoch [1/15], Training Loss: 0.5760, Training Accuracy: 0.7057, Training F1: 0.7053



Validation: 100%|██████████| 6/6 [00:00<00:00,  6.25it/s]


...

Epoch [15/15], Training Loss: 0.0088, Training Accuracy: 0.9990, Training F1: 0.9990



Validation: 100%|██████████| 6/6 [00:01<00:00,  4.93it/s]


Validation Loss: 0.7880, Validation Accuracy: 0.7333, Validation F1 Score: 0.7300
Model saved at checkpoints/saved_model.pth
ResNet18 model saved to output/models/resnet18_sgd.pth

Test score:



Evaluating: 100%|██████████| 7/7 [00:02<00:00,  3.42it/s]


Accuracy: 0.7500
F1 Score (Macro): 0.7490


### With Adam

In [50]:
resnet18_adam = get_resnet18_np(num_classes=len(class_names)).to(device)
optimizer_adam = torch.optim.Adam(resnet18_adam.parameters(), lr=0.0001)

criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=resnet18_adam,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_adam,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(resnet18_adam.state_dict(), os.path.join(model_dir, "resnet18_adam.pth"))
print(f"ResNet18 model saved to {os.path.join(model_dir, 'resnet18_adam.pth')}")

# Test the model
print("\nTest score:")
test(resnet18_adam, test_data_path=test_dir, device=device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)

Epoch 1/15: 100%|██████████| 90/90 [00:18<00:00,  4.75it/s, loss=0.225]


Epoch [1/15], Training Loss: 0.3282, Training Accuracy: 0.8617, Training F1: 0.8617



Validation: 100%|██████████| 6/6 [00:00<00:00,  6.02it/s]


Validation Loss: 1.2510, Validation Accuracy: 0.6278, Validation F1 Score: 0.5720
Model saved at checkpoints/saved_model.pth



Epoch 2/15: 100%|██████████| 90/90 [00:19<00:00,  4.59it/s, loss=0.082]


...

Epoch [15/15], Training Loss: 0.0160, Training Accuracy: 0.9923, Training F1: 0.9923


Validation: 100%|██████████| 6/6 [00:00<00:00,  6.20it/s]


Validation Loss: 1.1748, Validation Accuracy: 0.7167, Validation F1 Score: 0.7167
Model saved at checkpoints/saved_model.pth
ResNet18 model saved to output/models/resnet18_adam.pth

Test score:


Evaluating: 100%|██████████| 7/7 [00:01<00:00,  4.16it/s]

Accuracy: 0.7098
F1 Score (Macro): 0.7098


<h1>VGG16

<h2> Pretrained

In [51]:
def get_vgg16_pretrained(num_classes, transfer_learning=True):
    model = models.vgg16(pretrained=transfer_learning)
    if transfer_learning:
        for param in model.features.parameters():
            param.requires_grad = False
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    return model

### With SGD

In [52]:
# VGG16 Training
print("Training VGG16...")

vgg16_pretrained_sgd = get_vgg16_pretrained(num_classes=len(class_names)).to(device)

optimizer_sgd = torch.optim.SGD(vgg16_pretrained_sgd.parameters(), lr=0.0005, momentum=0.9)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=vgg16_pretrained_sgd,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_sgd,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(vgg16_pretrained_sgd.state_dict(), os.path.join(model_dir, "vgg16_pretrained_sgd.pth"))
print(f"VGG16 model saved to {os.path.join(model_dir, 'vgg16_pretrained_sgd.pth')}")

# Test the model
print("\nTest score:")
test(vgg16_pretrained_sgd, test_data_path=test_dir, device=device)


Training VGG16...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 96.1MB/s]
Epoch 1/15: 100%|██████████| 90/90 [00:21<00:00,  4.19it/s, loss=0.516]


Epoch [1/15], Training Loss: 0.5387, Training Accuracy: 0.7266, Training F1: 0.7265


Validation: 100%|██████████| 6/6 [00:01<00:00,  5.14it/s]


Validation Loss: 0.3212, Validation Accuracy: 0.8667, Validation F1 Score: 0.8659
Model saved at checkpoints/saved_model.pth


...

Epoch [15/15], Training Loss: 0.0695, Training Accuracy: 0.9759, Training F1: 0.9759


Validation: 100%|██████████| 6/6 [00:01<00:00,  4.47it/s]


Validation Loss: 0.2851, Validation Accuracy: 0.9056, Validation F1 Score: 0.9052
Model saved at checkpoints/saved_model.pth
VGG16 model saved to output/models/vgg16_pretrained_sgd.pth

Test score:


Evaluating: 100%|██████████| 7/7 [00:02<00:00,  3.24it/s]

Accuracy: 0.8661
F1 Score (Macro): 0.8661


### With Adam

In [53]:
print("Training VGG16...")

vgg16_pretrained_adam = get_vgg16_pretrained(num_classes=len(class_names)).to(device)

optimizer_adam = torch.optim.Adam(vgg16_pretrained_adam.parameters(), lr=0.00001)

criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=vgg16_pretrained_adam,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_adam,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(vgg16_pretrained_adam.state_dict(), os.path.join(model_dir, "vgg16_pretrained_adam.pth"))
print(f"VGG16 model saved to {os.path.join(model_dir, 'vgg16_pretrained_adam.pth')}")

# Test the model
print("\nTest score:")
test(vgg16_pretrained_adam, test_data_path=test_dir, device=device)


Training VGG16...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/15: 100%|██████████| 90/90 [00:21<00:00,  4.17it/s, loss=0.534]


Epoch [1/15], Training Loss: 0.6072, Training Accuracy: 0.6631, Training F1: 0.6631


Validation: 100%|██████████| 6/6 [00:01<00:00,  5.06it/s]


Validation Loss: 0.4341, Validation Accuracy: 0.8111, Validation F1 Score: 0.8077
Model saved at checkpoints/saved_model.pth


...

Epoch [15/15], Training Loss: 0.0125, Training Accuracy: 0.9986, Training F1: 0.9986


Validation: 100%|██████████| 6/6 [00:01<00:00,  5.15it/s]


Validation Loss: 0.3478, Validation Accuracy: 0.8944, Validation F1 Score: 0.8943
Model saved at checkpoints/saved_model.pth
VGG16 model saved to output/models/vgg16_pretrained_adam.pth

Test score:


Evaluating: 100%|██████████| 7/7 [00:02<00:00,  3.21it/s]

Accuracy: 0.8750
F1 Score (Macro): 0.8749


<h2>Not-pretrained

In [54]:
def get_vgg16_np(num_classes, transfer_learning=False):
    model = models.vgg16(pretrained=False)  # Do not load pre-trained weights
    ''''if transfer_learning:
        for param in model.features.parameters():
            param.requires_grad = False  # Freeze all layers of 'features' if transfer learning'''
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    return model

### With SGD

In [55]:
print("Training VGG16 (without pretrained weights)...")

# Initialize the VGG16 model without pretrained weights
vgg16_sgd_np = get_vgg16_np(num_classes=len(class_names)).to(device)

# Define optimizers
optimizer_sgd = torch.optim.SGD(vgg16_sgd_np.parameters(), lr=0.0005, momentum=0.9)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=vgg16_sgd_np,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_sgd,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(vgg16_sgd_np.state_dict(), os.path.join(model_dir, "vgg16_sgd_np.pth"))
print(f"VGG16 model saved to {os.path.join(model_dir, 'vgg16_sgd_np.pth')}")

# Test the model
print("\nTest score:")
test(vgg16_sgd_np, test_data_path=test_dir, device=device)


Training VGG16 (without pretrained weights)...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/15: 100%|██████████| 90/90 [00:28<00:00,  3.19it/s, loss=0.698]


Epoch [1/15], Training Loss: 0.6904, Training Accuracy: 0.5157, Training F1: 0.4185


Validation: 100%|██████████| 6/6 [00:01<00:00,  5.23it/s]


Validation Loss: 0.6848, Validation Accuracy: 0.6000, Validation F1 Score: 0.5238
Model saved at checkpoints/saved_model.pth


...

Epoch [15/15], Training Loss: 0.4764, Training Accuracy: 0.7716, Training F1: 0.7713


Validation: 100%|██████████| 6/6 [00:01<00:00,  5.31it/s]


Validation Loss: 0.4868, Validation Accuracy: 0.7833, Validation F1 Score: 0.7822
Model saved at checkpoints/saved_model.pth
VGG16 model saved to output/models/vgg16_sgd_np.pth

Test score:


Evaluating: 100%|██████████| 7/7 [00:02<00:00,  3.31it/s]

Accuracy: 0.7232
F1 Score (Macro): 0.7200


### With Adam

In [56]:
print("Training VGG16 (without pretrained weights)...")

# Initialize the VGG16 model without pretrained weights
vgg16_adam_np = get_vgg16_np(num_classes=len(class_names)).to(device)

optimizer_adam = torch.optim.Adam(vgg16_adam_np.parameters(), lr=0.0005)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=vgg16_adam_np,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_adam,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(vgg16_adam_np.state_dict(), os.path.join(model_dir, "vgg16_adam_np.pth"))
print(f"VGG16 model saved to {os.path.join(model_dir, 'vgg16_adam_np.pth')}")

# Test the model
print("\nTest score:")
test(vgg16_adam_np, test_data_path=test_dir, device=device)


Training VGG16 (without pretrained weights)...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/15: 100%|██████████| 90/90 [00:29<00:00,  3.01it/s, loss=0.761]


Epoch [1/15], Training Loss: 1.0683, Training Accuracy: 0.5024, Training F1: 0.5019


Validation: 100%|██████████| 6/6 [00:01<00:00,  5.15it/s]


Validation Loss: 0.6797, Validation Accuracy: 0.5000, Validation F1 Score: 0.3333
Model saved at checkpoints/saved_model.pth


...

Epoch [15/15], Training Loss: 0.0897, Training Accuracy: 0.9717, Training F1: 0.9717


Validation: 100%|██████████| 6/6 [00:01<00:00,  4.39it/s]


Validation Loss: 1.6648, Validation Accuracy: 0.7222, Validation F1 Score: 0.7221
Model saved at checkpoints/saved_model.pth
VGG16 model saved to output/models/vgg16_adam_np.pth

Test score:


Evaluating: 100%|██████████| 7/7 [00:02<00:00,  3.10it/s]

Accuracy: 0.6741
F1 Score (Macro): 0.6739
